In [10]:
import numpy as np
import os
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Conv1D, Dropout, Flatten

In [11]:
path = os.path.join(os.getcwd()+'/datasets/parkinsons.csv')
dataframe = pd.read_csv(path)
dataframe.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [72]:
from sklearn.utils import shuffle
data = shuffle(dataframe, random_state=101)
x = data.drop(columns=['name', 'status', 'MDVP:Jitter(Abs)', 'MDVP:RAP'])
y = data['status']
y.head()

39     1
163    1
61     0
32     0
123    1
Name: status, dtype: int64

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.40, random_state=101)

In [74]:
batch_size = 100
num_classes = 2
epochs = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [75]:
X_train = X_train.as_matrix(columns=None).reshape(X_train.shape + (1,))
X_test = X_test.as_matrix(columns=None).reshape(X_test.shape + (1,))

In [76]:
y_test.shape

(78, 2)

In [80]:
model = Sequential()
model.add(Conv1D(32, (3), activation='softmax', input_shape=[20, 1]))
model.add(Conv1D(64, (3), activation='softmax'))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(500, activation='softmax'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

In [81]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [82]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 117 samples, validate on 78 samples
Epoch 1/10
117/117 [==============================] - 1s - loss: 0.6916 - acc: 0.7179 - val_loss: 0.6904 - val_acc: 0.7949
Epoch 2/10
117/117 [==============================] - 0s - loss: 0.6911 - acc: 0.7265 - val_loss: 0.6895 - val_acc: 0.7949
Epoch 3/10
117/117 [==============================] - 0s - loss: 0.6903 - acc: 0.7265 - val_loss: 0.6886 - val_acc: 0.7949
Epoch 4/10
117/117 [==============================] - 0s - loss: 0.6896 - acc: 0.7265 - val_loss: 0.6876 - val_acc: 0.7949
Epoch 5/10
117/117 [==============================] - 0s - loss: 0.6887 - acc: 0.7265 - val_loss: 0.6868 - val_acc: 0.7949
Epoch 6/10
117/117 [==============================] - 0s - loss: 0.6882 - acc: 0.7265 - val_loss: 0.6857 - val_acc: 0.7949
Epoch 7/10
117/117 [==============================] - 0s - loss: 0.6876 - acc: 0.7265 - val_loss: 0.6850 - val_acc: 0.7949
Epoch 8/10
117/117 [==============================] - 0s - loss: 0.6868 - acc: 0.7265 - val_lo

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model_yaml = model.to_yaml()
with open("parkinsons.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")